## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-23-21-52-29 +0000,nypost,Israel says it will begin Gaza City military t...,https://nypost.com/2025/08/23/world-news/israe...
1,2025-08-23-21-51-41 +0000,bbc,Columbia University student among five dead in...,https://www.bbc.com/news/articles/cm2kp312ryxo...
2,2025-08-23-21-40-33 +0000,nypost,Alleged Queens carjacker suffers apparent heal...,https://nypost.com/2025/08/23/us-news/alleged-...
3,2025-08-23-21-35-11 +0000,bbc,UK's Chagos Islands deal 'significant victory'...,https://www.bbc.com/news/articles/cx2p1k1r2glo...
4,2025-08-23-21-31-00 +0000,wsj,The Pentagon has for months been blocking Ukra...,https://www.wsj.com/politics/national-security...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2307/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
57,trump,39
537,menendez,16
16,new,14
538,parole,10
751,lyle,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
68,2025-08-23-13-00-00 +0000,wsj,When President Trump called Gavin Newsom “News...,https://www.wsj.com/politics/policy/to-fight-t...,89
74,2025-08-23-12-29-00 +0000,wsj,The Texas Senate approved a redistricting plan...,https://www.wsj.com/politics/policy/texas-cong...,85
135,2025-08-23-03-00-00 +0000,wsj,Prospective ICE agents are going through a gru...,https://www.wsj.com/politics/policy/we-spent-s...,81
180,2025-08-22-22-15-00 +0000,nyt,Trump Says Chicago and New York Are Next for F...,https://www.nytimes.com/2025/08/22/us/politics...,70
151,2025-08-23-01-00-00 +0000,wsj,The Trump administration has a new weapon at i...,https://www.wsj.com/politics/policy/mortgage-f...,69


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
68,89,2025-08-23-13-00-00 +0000,wsj,When President Trump called Gavin Newsom “News...,https://www.wsj.com/politics/policy/to-fight-t...
128,68,2025-08-23-03-46-41 +0000,cbc,Lyle Menendez denied parole in parents' 1989 m...,https://www.cbc.ca/news/world/lyle-menendez-de...
150,43,2025-08-23-01-00-00 +0000,wsj,Texas’ redistricting plan has set off a nation...,https://www.wsj.com/politics/a-partisan-shot-f...
152,39,2025-08-23-01-00-00 +0000,wsj,Target’s New CEO Already Has Critics. He Bets ...,https://www.wsj.com/business/retail/target-new...
110,29,2025-08-23-07-07-23 +0000,nypost,Family of missing Texas woman Kaura Taylor who...,https://nypost.com/2025/08/23/world-news/texas...
158,29,2025-08-23-00-30-29 +0000,nypost,Pentagon fires head of intel agency responsibl...,https://nypost.com/2025/08/22/us-news/pentagon...
12,27,2025-08-23-20-43-29 +0000,nypost,Zohran Mamdani posts cheeky nod to ex-Adams ad...,https://nypost.com/2025/08/23/us-news/zohran-m...
38,27,2025-08-23-18-06-27 +0000,nypost,5 victims of deadly upstate NY bus crash ident...,https://nypost.com/2025/08/23/us-news/5-victim...
72,26,2025-08-23-12-42-42 +0000,cbc,U.S. seeks to deport Kilmar Abrego Garcia to U...,https://www.cbc.ca/news/world/kilmar-abrego-ga...
0,24,2025-08-23-21-52-29 +0000,nypost,Israel says it will begin Gaza City military t...,https://nypost.com/2025/08/23/world-news/israe...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
